In [1]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("playsplitLorem")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [2]:
lines = spark_context.textFile("hdfs://192.168.2.87:9000/lorem-split.txt")
print(lines.first())




"Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, 


In [3]:
# release the cores for another application!
# spark_context.stop()

In [6]:
from operator import itemgetter

lorem_split = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.87:9000/lorem-split.txt',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text',
    conf={'textinputformat.record.delimiter': 'abcd\n'}
)

lorem_split.take(4)


# (retrosheet
#     .filter(itemgetter(1))
#     .values()
#     .filter(lambda x: x)
#     .map(lambda v: (
#         v if v.startswith('id') else 'id,{0}'.format(v)).splitlines()))

[(0,
  '"Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, \ntotam rem aperiam,\neaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo.\n'),
 (223,
  'Nemo enim ipsam voluptatem quia voluptas sit\n aspernatur aut odit aut fugit, sed\n  quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. \n'),
 (387,
  'Neque porro quisquam est, qui dolorem ipsum\n quia dolor sit amet, consectetur, adipisci velit, sed quia\n  non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim \n'),
 (601,
  'ad minima veniam, quis\n nostrum exercitationem ullam corporis suscipit laboriosam\n , nisi ut aliquid ex ea commodi consequatur? \n')]